# Generating Clinical output

In [1]:
""" Generates Matrix containing for all the patients the available clinical data.

1rt column is Patient_ID
1rt line is headers
"""


import os
import glob
import math
from collections import defaultdict


import numpy
import pandas
pandas.set_option('display.max_columns',200)

DATA_CLI_PATH = os.path.join('../HEGP')
SAVE_CLI_PATH=os.path.join('../Data_output')
data_cl=pandas.read_csv(os.path.join(DATA_CLI_PATH,'SITC_HEGP_ImmunoS.txt'), sep="\t")

data_cl_Immunoscore=data_cl[['OfficialID','AnapathID','CD3.CT.score','CD3.IM.score','CD8.CT.score','CD8.IM.score','meanScore','immunoScore5gr','immunoScore3gr']]
data_cl_Clinical=data_cl[['OfficialID','AnapathID','gender','age','race','dx_date','fu_stat','dth_date','cause','ltf__date','rc_stat','rc_date','pg_stat',
	'pg_date','np_date','evl_date','t_stage','t_stage_mod_6is4_7is1','n_stage','plnode','m_stage','UICC_TNM','preop_chemo','postop_chemo','postop_biotherapy',
    'surg_pt_date','resect_pt','surg_pt_type','nlnode','plnode','colon','sidedness','cecum','ascending','splenflex','transverse','hepflex','descending',
    'sigmoid','immuno_tx','differentiation','mucinous_colloide','occlusion','perforation','venous_emboli','lymphatic_invasion','perineural_invasion','msi_ihc',
    'msi_gen','lynch_syndrome','fap','ibs','tumor_budding','cimp','p53_status','kras_status','apc_status','braf_status','pi3k_status','MSI_NBP']]



## Survival Data

Calculating TTR: Time to Relapse
Calculating OS: Outcome survival
In days using the python function datetime

In [2]:
data_cl_Survival=data_cl[['OfficialID','AnapathID','dx_date','fu_stat','dth_date','cause','ltf__date','rc_stat','rc_date']]
from datetime import datetime
date_format = "%m/%d/%Y"

# For TTR, if rc_stat=1 the relapse has occured, time is rc_date-dx-date otherwise = ltf_date-dx_date
# For OS if fu_stat=1 the patient Died, time is dth_date-dx_date  otherwise = ltf_date-dx_date
data_cl_Survival.set_index('OfficialID', drop=False,inplace=True)
for patient_id in list(data_cl['OfficialID']):
    start_date=datetime.strptime(data_cl_Survival.loc[patient_id,'dx_date'], date_format)
    #TTR
    if data_cl_Survival.loc[patient_id,'rc_stat']==1:
        end_date=datetime.strptime(data_cl_Survival.loc[patient_id,'rc_date'], date_format)
        data_cl_Survival.loc[patient_id,'TTR_2012_Days']=(end_date-start_date).days
    else:
        end_date=datetime.strptime(data_cl_Survival.loc[patient_id,'ltf__date'], date_format)
        data_cl_Survival.loc[patient_id,'TTR_2012_Days']=(end_date-start_date).days
     #OS
    if data_cl_Survival.loc[patient_id,'fu_stat']==1:
        end_date=datetime.strptime(data_cl_Survival.loc[patient_id,'dth_date'], date_format)
        data_cl_Survival.loc[patient_id,'OS_2012_Days']=(end_date-start_date).days
    else:
        end_date=datetime.strptime(data_cl_Survival.loc[patient_id,'ltf__date'], date_format)
        data_cl_Survival.loc[patient_id,'OS_2012_Days']=(end_date-start_date).days


C:\Users\HEGP\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\HEGP\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## Saving the Dataframes

In [3]:
data_cl_Survival.head()



,OfficialID,AnapathID,dx_date,fu_stat,dth_date,cause,ltf__date,rc_stat,rc_date,TTR_2012_Days,OS_2012_Days
OfficialID,,,,,,,,,,,
PAR001,PAR001,c99H08,01/04/1999,1,07/15/2003,0.0,07/15/2003,0,NaN,1653.0,1653.0
PAR002,PAR002,c98H2501,05/18/1998,1,06/12/1998,0.0,06/12/1998,0,NaN,25.0,25.0
PAR003,PAR003,c93H844,12/27/1992,1,07/06/1993,NaN,07/06/1993,1,02/19/1993,54.0,191.0
PAR004,PAR004,c99H5143,11/08/1999,1,09/09/2005,0.0,09/09/2005,0,NaN,2132.0,2132.0
PAR005,PAR005,c01H4447,06/06/2001,0,NaN,NaN,07/20/2007,0,NaN,2235.0,2235.0


In [4]:
data_cl_Immunoscore.to_csv(os.path.join(SAVE_CLI_PATH,'data_HEGP_Immunoscore_original.txt'),sep=';',index=None)
data_cl_Survival.to_csv(os.path.join(SAVE_CLI_PATH,'data_HEGP_Survival.txt'),sep=';',index=None)
data_cl_Clinical.to_csv(os.path.join(SAVE_CLI_PATH,'data_HEGP_Clinical.txt'),sep=';',index=None)